In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize

from cct.consts import EV2KCALMOL
from cct.energy import CalculatorFactory
from cct.rdkit.conformers import embed

initial_energy_threshold = 5.0

calcs = CalculatorFactory()
enumerator = rdMolStandardize.TautomerEnumerator()

mol = Chem.MolFromSmiles("N1=CC=N[NH]1")
tauts = enumerator.Enumerate(mol)
tauts = list(tauts.tautomers)
charge = Chem.GetFormalCharge(tauts[0])

multiplicity = 1
optimised_tauts = []
energies = []
for taut in tauts:
    taut = Chem.AddHs(taut, addCoords=True)
    taut = embed(taut, num_confs=1)

    calcs._optimise(taut, method="GFN2-xTB", multiplicity=multiplicity, charge=charge)
    gas_energy = calcs._singlepoint(
        taut, method="EGRET1", multiplicity=multiplicity, charge=charge
    )
    solvation_energy = calcs.solvation_energy(
        taut, solvent="water", multiplicity=multiplicity, charge=charge
    )
    energy = gas_energy + solvation_energy
    energies.append(energy)
    optimised_tauts.append(taut)

energies = np.array(energies) * EV2KCALMOL
relative_energies = energies - np.min(energies)

low_energy_mask = relative_energies < initial_energy_threshold
low_energy_tauts = np.array(optimised_tauts)[low_energy_mask]

for taut in low_energy_tauts:
    mol = _generate_conformers_rdkit(
        taut, mode=mode, multiplicity=multiplicity, charge=charge, num_cores=num_cores
    )